In [1]:
import re
from collections import defaultdict, Counter, OrderedDict
from collections.abc import Mapping
from copy import copy, deepcopy
from dataclasses import dataclass, field, replace
from enum import Enum
from functools import partial, reduce
from itertools import combinations, product
from math import ceil, cos, floor, prod, radians, sin
from typing import Any, Dict, List, Tuple

import numpy as np

## Utils

In [2]:
def read(path):
    with open(path) as f: return f.read()

## Day 1

In [3]:
def split_ints(text):
    return [int(x) for x in text.splitlines()]

In [4]:
DAY01_TEST = """\
1721
979
366
299
675
1456
"""
DAY01_TEST = split_ints(DAY01_TEST)
DAY01_DATA = split_ints(read("data/day01.txt"))

In [5]:
def prod_combinations_where_sum_eq_2020(ints, n):
    return next(prod(x) for x in combinations(ints, r=n) if sum(x) == 2020)

In [6]:
assert prod_combinations_where_sum_eq_2020(DAY01_TEST, 2) == 514579
assert prod_combinations_where_sum_eq_2020(DAY01_DATA, 2) == 866436
assert prod_combinations_where_sum_eq_2020(DAY01_TEST, 3) == 241861950
assert prod_combinations_where_sum_eq_2020(DAY01_DATA, 3) == 276650720

## Day 2

In [7]:
def parse_policy_data(text):
    args = []
    for line in text.splitlines():
        low, high, char, password = re.match(r"^(\d+)-(\d+) ([a-z]): ([a-z]+)$", line).groups()
        args.append((int(low), int(high), char, password))
    return args

In [8]:
DAY02_TEST = """\
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
"""
DAY02_TEST = parse_policy_data(DAY02_TEST)
DAY02_DATA = parse_policy_data(read("data/day02.txt"))

In [9]:
def count_policy(low, high, char, password):
    return low <= password.count(char) <= high

In [10]:
def count_valid_passwords(policy_data, policy):
    return sum(policy(*p) for p in policy_data)

In [11]:
assert count_valid_passwords(DAY02_TEST, count_policy) == 2
assert count_valid_passwords(DAY02_DATA, count_policy) == 655

In [12]:
def position_policy(low, high, char, password):
    return sum(password[i-1] == char for i in (low, high)) == 1

In [13]:
assert count_valid_passwords(DAY02_TEST, position_policy) == 1
assert count_valid_passwords(DAY02_DATA, position_policy) == 673

## Day 3

In [14]:
def parse_trees(text):
    return np.array(
        [
            [char == "#" for char in line]
            for line in text.splitlines()
        ]
    )

In [15]:
DAY03_TEST = """\
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
"""
DAY03_TEST = parse_trees(DAY03_TEST)
DAY03_DATA = parse_trees(read("data/day03.txt"))

In [16]:
def count_trees(arr, dx, dy):
    ny, nx = arr.shape
    y = np.arange(0, ny, dy)
    x = np.mod(np.arange(0, len(y)) * dx, nx)
    return np.sum(arr[y, x])

In [17]:
assert count_trees(DAY03_TEST, 3, 1) == 7
assert count_trees(DAY03_DATA, 3, 1) == 216

In [18]:
SLOPES = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]

def prod_count_trees_for_slopes(lines):
    return prod(count_trees(lines, dx, dy) for dx, dy in SLOPES)

In [19]:
assert prod_count_trees_for_slopes(DAY03_TEST) == 336
assert prod_count_trees_for_slopes(DAY03_DATA) == 6708199680

## Day 4

In [20]:
def parse_passports(text):
    return [
        dict(key_val.split(":") for key_val in passport.split())
        for passport in text.split("\n\n")
    ]

In [21]:
DAY04_TEST = """\
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
"""
DAY04_TEST = parse_passports(DAY04_TEST)
DAY04_DATA = parse_passports(read("data/day04.txt"))

In [22]:
REQUIRED = set("byr iyr eyr hgt hcl ecl pid".split())

def is_passport_keys_valid(p):
    return set(p.keys()).issuperset(REQUIRED)

In [23]:
def count_valid_passports(passports, is_valid):
    return sum(is_valid(p) for p in passports)

In [24]:
assert count_valid_passports(DAY04_TEST, is_passport_keys_valid) == 2
assert count_valid_passports(DAY04_DATA, is_passport_keys_valid) == 210

In [25]:
VALID_ECL = set("amb blu brn gry grn hzl oth".split())

def is_passport_valid(p):
    if not is_passport_keys_valid(p): return False
    if not (match := re.match("^(\d+)(cm|in)$", p["hgt"])): return False
    height, unit = match.groups()
    height = int(height)
    valid = {
        "byr": 1920 <= int(p["byr"]) <= 2002,
        "iyr": 2010 <= int(p["iyr"]) <= 2020,
        "eyr": 2020 <= int(p["eyr"]) <= 2030,
        "hgt": (150 <= height <= 193 if unit == "cm" else 59 <= height <= 76),
        "hcl": re.match("^\#[0-9a-f]{6}$", p["hcl"]) is not None,
        "ecl": p["ecl"] in VALID_ECL,
        "pid": re.match("^\d{9}$", p["pid"]) is not None
    }
    return all(valid.values())

In [26]:
assert count_valid_passports(DAY04_DATA, is_passport_valid) == 131

## Day 5

In [27]:
def decode(chars, low, up):
    for char in chars:
        mid = (low + up) / 2
        if char in "FL":
            up  = floor(mid)
        else:
            low = ceil(mid)
    return low

def parse_seat_ids(text):
    seat_ids = []
    for line in text.splitlines():
        row = decode(line[:7], 0, 127)
        col = decode(line[7:], 0, 7)
        seat_ids.append(8 * row + col)
    return seat_ids

In [28]:
DAY05_TEST = """\
FBFBBFFRLR
BFFFBBFRRR
FFFBBBFRRR
BBFFBBFRLL
"""
DAY05_TEST = parse_seat_ids(DAY05_TEST)
DAY05_DATA = parse_seat_ids(read("data/day05.txt"))

In [29]:
assert max(DAY05_DATA) == 913
assert set(range(min(DAY05_DATA), max(DAY05_DATA) + 1)) - set(DAY05_DATA) == {717}

In [30]:
def parse_seat_ids_binary(text):
    seat_ids = []
    for line in text.split():
        row = line[:7].replace("B", "1").replace("F", "0")
        seat = line[7:].replace("R", "1").replace("L", "0")
        seat_id = 8 * int(row, 2) + int(seat, 2)
        seat_ids.append(seat_id)
    return seat_ids

## Day 6

In [31]:
def parse_customs_forms(text):
    return [
        [set(x) for x in group.split("\n")]
        for group in text.strip().split("\n\n")
    ]

In [32]:
DAY06_TEST = """\
abc

a
b
c

ab
ac

a
a
a
a

b
"""
DAY06_TEST = parse_customs_forms(DAY06_TEST)
DAY06_DATA = parse_customs_forms(read("data/day06.txt"))

In [33]:
def count_answers(forms, set_agg):
    return sum(len(set_agg(*x)) for x in forms)

In [34]:
assert count_answers(DAY06_TEST, set.union) == 11
assert count_answers(DAY06_DATA, set.union) == 6930
assert count_answers(DAY06_TEST, set.intersection) == 6
assert count_answers(DAY06_DATA, set.intersection) == 3585

## Day 7

In [35]:
def parse_bags(text):
    containers = defaultdict(dict)
    for line in text.splitlines():
        container, contained_str = line.split(" bags contain ")
        if contained_str.startswith("no"):
            continue
        for s in re.split(" bags?(?:, |.)", contained_str)[:-1]:
            if s.startswith("no"):
                continue
            weight, contained = s.split(" ", 1)
            containers[contained][container] = int(weight)
    return containers

In [36]:
DAY07_TEST = """\
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""
DAY07_TEST = parse_bags(DAY07_TEST)
DAY07_DATA = parse_bags(read("data/day07.txt"))

In [37]:
def descendants(edges, root, visited=set()):
    if root not in edges: return visited
    visited = visited.union(edges[root].keys())
    return set.union(visited, *(descendants(edges, child, visited) for child in edges[root]))

In [38]:
def count_containing_bags(contains, color):
    return len(descendants(contains, color))

In [39]:
COLOR = "shiny gold"
assert count_containing_bags(DAY07_TEST, COLOR) == 4
assert count_containing_bags(DAY07_DATA, COLOR) == 289

In [40]:
def reverse(edges):
    res = {}
    for parent, children in edges.items():
        for node, weight in children.items():
            if node not in res:
                res[node] = {}
            res[node].update({parent: weight})
    return res

In [41]:
def recursive_sum_weights(edges, root):
    if root not in edges: return 0
    return sum(
        weight * (1 + recursive_sum_weights(edges, child))
        for child, weight in edges[root].items()
    )

In [42]:
assert recursive_sum_weights(reverse(DAY07_TEST), COLOR) == 32
assert recursive_sum_weights(reverse(DAY07_DATA), COLOR) == 30055

## Day 8

In [43]:
def parse_ops(text):
    ops = []
    for l in text.splitlines():
        op, n = l.split()
        ops.append((op, int(n)))
    return ops

In [44]:
DAY08_TEST = """\
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
"""
DAY08_TEST = parse_ops(DAY08_TEST)
DAY08_DATA = parse_ops(read("data/day08.txt"))

In [45]:
@dataclass
class Program:
    ops: List[Tuple[str, int]]
    pos: int = 0
    acc: int = 0
    visited: set = field(default_factory=set)
    can_swap: bool = True

    SWAPS = {"nop": "jmp", "jmp": "nop"}

    def run(self):
        if self.pos in self.visited:
            return False, self.acc
        if self.pos >= len(self.ops):
            return True, self.acc
        op, n = self.ops[self.pos]
        if self.can_swap and op in self.SWAPS and (result := self.swap().run()) and result[0]:
            return result
        self.visited = {*self.visited, self.pos}
        return {"nop": self.nop, "acc": self.accumulate, "jmp": self.jump}[op](n).run()

    def nop(self, n):
        return replace(self, pos=self.pos+1)

    def accumulate(self, n):
        return replace(self, pos=self.pos+1, acc=self.acc+n)

    def jump(self, n):
        return replace(self, pos=self.pos+n)

    def swap(self):
        op, n = self.ops[self.pos]
        ops = self.ops.copy()
        to_op = self.SWAPS[op]
        ops[self.pos] = (to_op, n)
        return replace(self, ops=ops, can_swap=False)

In [46]:
assert Program(DAY08_TEST, can_swap=False).run() == (False, 5)
assert Program(DAY08_DATA, can_swap=False).run() == (False, 2080)
assert Program(DAY08_TEST).run() == (True, 8)
assert Program(DAY08_DATA).run() == (True, 2477)

## Day 9

In [47]:
DAY09_TEST = """\
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
"""
DAY09_TEST = split_ints(DAY09_TEST)
DAY09_DATA = split_ints(read("data/day09.txt"))

In [48]:
def first(ints, window_size):
    end = window_size
    for x in ints[end:]:
        start = end - window_size
        try:
            next(True for y in combinations(ints[start:end], 2) if sum(y) == x)
        except StopIteration:
            return x
        end += 1

In [49]:
assert first(DAY09_TEST, 5) == 127

In [50]:
assert first(DAY09_DATA, 25) == 26134589

In [51]:
def calc_ranges(i):
    if i <= 1: return []
    return [(j, i) for j in range(1, i)]

In [52]:
assert calc_ranges(1) == []
assert calc_ranges(2) == [(1, 2)]
assert calc_ranges(3) == [(1, 3), (2, 3)]
assert calc_ranges(4) == [(1, 4), (2, 4), (3, 4)]

In [53]:
def func(ints, eq):
    for i, x in enumerate(ints):
        ranges = calc_ranges(i)
        for start, end in ranges:
            s = sum(ints[start:end])
            if s == eq:
                return min(ints[start:end]) + max(ints[start:end])
    assert False

In [54]:
assert func(DAY09_TEST, 127) == 62

In [55]:
assert func(DAY09_DATA, 26134589) == 3535124

## Day 10

In [56]:
DAY10_TEST1 = """\
16
10
15
5
1
11
7
19
6
12
4
"""
DAY10_TEST1 = split_ints(DAY10_TEST1)
DAY10_TEST2 = """\
28
33
18
42
31
14
46
20
48
47
24
23
49
45
19
38
39
11
1
32
25
35
8
17
7
9
4
2
34
10
3
"""
DAY10_TEST2 = split_ints(DAY10_TEST2)
DAY10_DATA = split_ints(read("data/day10.txt"))

In [57]:
def count_jolt_diff(ints):
    ints = sorted([0] + ints + [max(ints) + 3])
    diffs = [b - a for a, b in zip(ints, ints[1:])]
    counts = Counter(diffs)
    return counts[1] * counts[3]

In [58]:
assert count_jolt_diff(DAY10_TEST1) == 7 * 5

In [59]:
assert count_jolt_diff(DAY10_TEST2) == 22 * 10

In [60]:
assert count_jolt_diff(DAY10_DATA) == 1984

In [61]:
def count_paths(ints, pos=0, paths=1):
    if pos >= len(ints):
        return paths
    
    if pos < len(ints) - 2:
        diff = ints[pos+2] - ints[pos]
        if diff <= 3:
            paths = count_paths(ints, pos+2, paths+1)

    if pos < len(ints) - 3:
        diff = ints[pos+3] - ints[pos]
        if diff <= 3:
            paths = count_paths(ints, pos+3, paths+1)

    paths = count_paths(ints, pos+1, paths)

    return paths

In [62]:
def func(ints):
    ints = [0] + sorted(ints) + [max(ints) + 3]

    splits = []
    last = 0
    for i, _ in enumerate(ints):
        if ints[i] - ints[i-1] >= 3:
            splits.append(ints[last:i])
            last = i
            
    return prod(count_paths(s) for s in splits)

In [63]:
assert func(DAY10_TEST1) == 8

In [64]:
assert func(DAY10_TEST2) == 19208

In [65]:
assert func(DAY10_DATA) == 3543369523456

## Day 11

In [66]:
def split_array(text):
    return np.array([[CHAR_TO_INT[c] for c in l] for l in text.strip().split("\n")])

In [67]:
class Seat(Enum):
    EMPTY = "L"
    OCCUPIED = "#"

In [68]:
@dataclass
class SeatMatrix(Mapping):
    seats: Dict[Tuple[int, int], Seat]
    shape: Tuple[int, int]

    def __repr__(self):
        res = ""
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                seat = self.seats.get((i, j))
                res += seat.value if seat else "."
            res += "\n"
        return res

    def __getitem__(self, key):
        return self.seats[key]

    def __iter__(self):
        return iter(self.seats)

    def __len__(self):
        return len(self.seats)

In [69]:
def parse_seats(text):
    seats = {}
    for x, line in enumerate(text.split()):
        for y, char in enumerate(line):
            if char != ".":
                seats[x, y] = Seat(char)
    shape = max(x for x, _ in seats.keys()), max(y for y, _ in seats.keys())
    return SeatMatrix(seats, shape)

In [70]:
DAY11_TEST = """\
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""
DAY11_TEST = parse_seats(DAY11_TEST)
DAY11_DATA = parse_seats(read("data/day11.txt"))

In [71]:
DIRECTIONS = [(dx, dy) for dx, dy in product((-1, 0, 1), repeat=2) if (dx, dy) != (0, 0)]

def rays(seats, i, j, length=1):
    for dx, dy in DIRECTIONS:
        x, y = i, j
        for _ in range(length):
            x += dx
            y += dy
            if x < 0 or x >= seats.shape[0] or y < 0 or y >= seats.shape[1]:
                break
            seat = seats.get((x, y))
            if seat:
                yield seat
                break

In [72]:
def evolve(seats):
    res = seats.copy()
    for (i, j), seat in seats.items():
        n_occupied = sum(n == Seat.OCCUPIED for n in rays(seats, i, j, length=1))
        if seat == Seat.EMPTY and n_occupied == 0:
            res[(i, j)] = Seat.OCCUPIED
        elif seat == Seat.OCCUPIED and n_occupied >= 4:
            res[(i, j)] = Seat.EMPTY
    return res

In [73]:
def func(seats):
    prev = None
    while not np.all(prev == seats):
        prev = seats
        seats = evolve(seats)
    return sum(s == Seat.OCCUPIED for s in seats)

In [74]:
def print_seats(seats, shape):
    for i in range(shape[0]):
        for j in range(shape[1]):
            seat = seats.get((i, j))
            seat = seat.value if seat else "."
            print(seat, end="")
        print()

In [75]:
# evolve(DAY11_TEST)

In [76]:
def count_occ(arr, i, j):
    n_occ = 0
    for dx, dy in SLOPES:
        xi, yi = i, j
        while True:
            xi, yi = xi + dx, yi + dy
            if xi >= arr.shape[0] or xi < 0 or yi >= arr.shape[1] or yi < 0:
                break
            if arr[xi, yi] == OCC:
                n_occ += 1
            if arr[xi, yi] != FLOOR:
                break
    return n_occ

In [77]:
def evolve2(arr):
    res = arr.copy()
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            x = res[i, j]
            if x == FLOOR:
                continue
            n = rays(arr, i, j)
            n_occ = sum(arr[x, y] == OCC for x, y in n)
            if x == EMPTY:
                if not n_occ:
                    res[i, j] = OCC
            elif x == OCC:
                if n_occ >= 5:  # 4 (neighbors) + 1 (current)
                    res[i, j] = EMPTY
    return res

In [78]:
# arr = DAY11_DATA.copy()
# prev = None
# while not np.all(prev == arr):
#     prev = arr
#     arr = evolve2(arr)
# assert np.sum(arr == OCC) == 1978

## Day 12

In [79]:
def parse_instructions(text):
    return [(l[0], int(l[1:])) for l in text.splitlines()]

In [80]:
DAY12_TEST = """\
F10
N3
F7
R90
F11
"""
DAY12_TEST = parse_instructions(DAY12_TEST)
DAY12_DATA = parse_instructions(read("data/day12.txt"))

In [81]:
def rotate_ccw(x, y, deg):
    rad = radians(deg)
    x2 = x * cos(rad) - y * sin(rad)
    y2 = x * sin(rad) + y * cos(rad)
    return int(round(x2)), int(round(y2))

def rotate_cw(x, y, deg):
    return rotate_ccw(x, y, -deg)

In [82]:
@dataclass
class Navigator:
    dx: int
    dy: int
    x: int = 0
    y: int = 0

    def navigate(self, instructions):
        nav = self
        for code, n in instructions:
            nav = {
                "F": nav.forward,
                "N": nav.north,
                "E": nav.east,
                "S": nav.south,
                "W": nav.west,
                "L": nav.left,
                "R": nav.right,
            }[code](n)
        return abs(nav.x) + abs(nav.y)

    def forward(self, n):
        return replace(self, x=self.x+self.dx*n, y=self.y+self.dy*n)

    def north(self, n):
        return replace(self, y=self.y+n)

    def east(self, n):
        return replace(self, x=self.x+n)

    def south(self, n):
        return replace(self, y=self.y-n)

    def west(self, n):
        return replace(self, x=self.x-n)

    def left(self, n):
        dx, dy = rotate_ccw(self.dx, self.dy, n)
        return replace(self, dx=dx, dy=dy)

    def right(self, n):
        dx, dy = rotate_cw(self.dx, self.dy, n)
        return replace(self, dx=dx, dy=dy)

In [83]:
assert Navigator(dx=1, dy=0).navigate(DAY12_TEST) == 25
assert Navigator(dx=1, dy=0).navigate(DAY12_DATA) == 521

In [84]:
class WaypointNavigator(Navigator):
    def north(self, n):
        return replace(self, dy=self.dy+n)

    def east(self, n):
        return replace(self, dx=self.dx+n)

    def south(self, n):
        return replace(self, dy=self.dy-n)

    def west(self, n):
        return replace(self, dx=self.dx-n)

In [85]:
assert WaypointNavigator(dx=10, dy=1).navigate(DAY12_TEST) == 286
assert WaypointNavigator(dx=10, dy=1).navigate(DAY12_DATA) == 22848

## Day 13

In [86]:
# DAY13_TEST = """
# """
# DAY13_TEST = split(DAY13_TEST)
# DAY13_DATA = split(read("data/day13.txt"))

## Day 14

In [87]:
DOCKING_PATTERNS = [
    ("mem", r"^mem\[(\d+)\] = (\d+)$"),
    ("mask", r"^mask = ([X01]{36})$"),
]

def parse_docking_program(text):
    program = []
    for line in text.splitlines():
        for key, pat in DOCKING_PATTERNS:
            if match := re.match(pat, line):
                program.append((key, *match.groups()))
                break
        else:
            raise ValueError(f"Could not parse line: '{line}'")
    return program

In [88]:
DAY14_TEST = """\
mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0
"""
DAY14_TEST = parse_docking_program(DAY14_TEST)
DAY14_DATA = parse_docking_program(read("data/day14.txt"))

In [89]:
def run_docking_program(program):
    mask = {}
    mem = {}
    for p in program:
        op = p[0]
        vals = p[1:]
        if op == "mask":
            mask_str = vals[0]
            mask = dict((i, x) for i, x in enumerate(mask_str) if x != "X")
        elif op == "mem":
            key = int(vals[0])
            val = int(vals[1])
            val_bin = format(int(val), "036b")    # 0-padded 36-length binary string
            masked_val_bin = "".join(mask.get(i) or x for i, x in enumerate(val_bin))
            masked_val = int(masked_val_bin, 2)
            mem[key] = masked_val
    return sum(mem.values())

In [90]:
assert run_docking_program(DAY14_TEST) == 165

In [91]:
assert run_docking_program(DAY14_DATA) == 11612740949946

In [92]:
DAY14_TEST2 = """\
mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1
"""
DAY14_TEST2 = parse_docking_program(DAY14_TEST2)

In [93]:
def run_docking_program2(program):
    mask_to_1 = []
    mask_to_floating = []
    mem = {}
    for p in program:
        op = p[0]
        vals = p[1:]
        if op == "mask":
            mask_str = vals[0]
            mask_to_1 = []
            mask_to_floating = []
            for i, x in enumerate(mask_str):
                if x == "1":   mask_to_1.append(i)
                elif x == "X": mask_to_floating.append(i)
        elif op == "mem":
            key = int(vals[0])
            val = int(vals[1])

            key_bin = list(format(key, "036b"))    # 0-padded 36-length binary string
            for i in mask_to_1:
                key_bin[i] = "1"

            for floating in product(["0", "1"], repeat=len(mask_to_floating)):
                for i, v in zip(mask_to_floating, floating):
                    key_bin[i] = v
                mem[int("".join(key_bin), 2)] = val

    return sum(mem.values())

In [94]:
assert run_docking_program2(DAY14_TEST2) == 208

In [95]:
assert run_docking_program2(DAY14_DATA) == 3394509207186